# NASA Common Research Model
## Drag Prediction Workshop 5

### Matched Lift Coefficient (0.50) with Drag Components


### References

https://aiaa-dpw.larc.nasa.gov/

https://aiaa-dpw.larc.nasa.gov/Workshop5/presentations/DPW5_Presentation_Files/14_DPW5%20Summary-Draft_V7.pdf

In [ ]:
CASE='dpw5_L1'
DATA_DIR='.'
REF_DATA_DIR='.'

## Define Data Location
For remote data the interaction will use ssh to securely interact with the data<br/>
This uses the reverse connection capability in paraview so that the paraview server can be submitted to a job scheduler<br/>
Note: The default paraview server connection will use port 11111

In [ ]:
import os
from zutil import analysis
analysis.data_init(default_data_dir=DATA_DIR,
                   default_ref_data_dir=REF_DATA_DIR)


### Initialise Environment

In [ ]:
from zutil.plot import *

from paraview.simple import *
paraview.simple._DisableFirstRenderCameraReset()
import math

### Get status file

In [ ]:
from zutil.post import get_status_dict
status=get_status_dict(CASE)
num_procs = str(status['num processor'])

### Define test conditions

In [ ]:
alpha = 2.21 # degrees
reference_area = 594720.0 # inches^2
reference_length = 275.8 # inches, mean chord. 
reference_span = 1156.75 # inches


In [ ]:
from zutil.post import cp_profile_wall_from_file

def plot_cp_profile(ax,file_root,span_loc, label):
    
    force_data = cp_profile_wall_from_file(file_root,
                              [0.0,1.0,0.0],
                              [0, span_loc*reference_span, 0],
                              func=plot_array,
                              axis=ax,
                              span_loc=span_loc,
                              alpha=alpha,
                              label=label)
    
def plot_array(data_array,pts_array,**kwargs):
    ax = kwargs['axis']
    span_loc = kwargs['span_loc']
    cp_array = data_array.GetPointData()['cp']
    chord_array = data_array.GetPointData()['chord']
    ax.plot(chord_array, cp_array , '.',label=kwargs['label'])

### Comparison Data

In [ ]:
# Reproduce plots from DPW5 presentation, page 45
# Pressure data points (reference semi-span: 1156.75) 
# Station Type          WBL      ETA     Chord
# 1       CFD Cut Only  121.459  0.1050  466.5
# 2       CFD Cut Only  133.026  0.1150  459.6
# 3       CFD Cut Only  144.594  0.1250  452.7
# 4       Pressure Tap  151.074  0.1306  448.8
# 5       Pressure Tap  232.444  0.2009  400.7
# 6       Pressure Tap  327.074  0.2828  345.0
# 7       CFD Cut Only  396.765  0.3430  304.1
# 8       CFD Cut Only  427.998  0.3700  285.8
# 9       Pressure Tap  459.370  0.3971  278.1
# 10      Pressure Tap  581.148  0.5024  248.3
# 11      Pressure Tap  697.333  0.6028  219.9
# 12      Pressure Tap  840.704  0.7268  184.8
# 13      Pressure Tap  978.148  0.8456  151.2
# 14      Pressure Tap  1098.126 0.9500  121.7
# 15      CFD Cut Only  1122.048 0.9700  116.0
# 16      CFD Cut Only  1145.183 0.9900  110.5

#eta_values = [0.1306, 0.2828, 0.3971, 0.5024, 0.7268, 0.9500] # stations 4, 6, 9, 10, 12, 14

from collections import OrderedDict
station_values = OrderedDict([("S04" , 0.1306), ("S06" , 0.2828), ("S09" , 0.3971), 
                              ("S10" , 0.5024), ("S12" , 0.7268), ("S14" , 0.9500)])
sources = [["Edge SST","r"], ["CFD++ SST","g"], ["FUN3D SA", "m"], ["MFlow SA", "y"]] 
dpw5_comparative_data = eval(open(os.path.join(analysis.data.ref_data_dir, 'DPW5_Comparative_Data.py'), 'r').read())


## Force and Residual Convergence

In [ ]:
from zutil.post import get_csv_data

def plot_residual(ax, csv_data):
    eqns = ['rho', 'rhoV[0]', 'rhoV[1]', 'rhoV[2]', 'rhoE', 'rhok', 'rhoOmega']
    cycles = csv_data['Cycle'].tolist()
    ax.set_yscale('log')
    for eqn in eqns:
        ax.plot(cycles, csv_data[eqn], label=eqn)

fig_list = {}
C_L = {}
C_D = {}
C_D_P = {}
C_D_F = {}
plots = ('Drag', 'Lift', 'Residuals DPW L1')
for plot in plots:
    fig_list[plot] = get_figure(plt)
    fig = fig_list[plot]
    ax = fig.gca()
    set_title(ax, plot+' vs Cycle\n')
    x_label(ax, 'Cycle')
    if plot.startswith('Res'):
        y_label(ax, 'Residual')
    else:
        y_label(ax, plot)
    
csv_file = os.path.join(analysis.data.data_dir, CASE+"_report.csv")
csv_data = get_csv_data(csv_file, delim='\s+', header=True)

drag = np.asarray(csv_data['wall_Ftx'].tolist())
lift = np.asarray(csv_data['wall_Ftz'].tolist())
friction = np.asarray(csv_data['wall_Ftfx'].tolist())
pressure = np.asarray(csv_data['wall_Ftpx'].tolist())
cycles = csv_data['Cycle'].tolist()

C_L[CASE] = np.mean(lift[-100:])
C_D[CASE] = np.mean(drag[-100:])
C_D_P[CASE] = np.mean(pressure[-100:])
C_D_F[CASE] = np.mean(friction[-100:])

for plot in plots:
    fig = fig_list[plot]
    ax = fig.gca()
    if plot == 'Drag':
        ax.axis([0, 10000, 0.005, 0.03])
        ax.plot(cycles, drag, label='$\mathbf{C_{d}}$ %s' % CASE)
        ax.plot(cycles, friction, label='$\mathbf{C_{df}}$ %s' % CASE)
        ax.plot(cycles, pressure, label='$\mathbf{C_{dp}}$ %s' % CASE)
    elif plot == 'Lift':
        ax.axis([0, 10000, 0.49, 0.51])
        ax.plot(cycles, lift, label='$\mathbf{C_l}$ %s' % CASE)
        ax.axhline(0.5, ls='dashed')
    elif plot.endswith('L1') and CASE == 'dpw5_L1':
        plot_residual(ax, csv_data) 
    elif plot.endswith('L2') and CASE == 'dpw5_L2':
        plot_residual(ax, csv_data) 


for plot in plots:
    fig = fig_list[plot]
    ax = fig.gca()
    set_legend(ax, 'best')
    #fig.savefig(os.path.join("output", "DPW5_"+plot+"_convergence.png"))

            

## Cp Profile

In [ ]:
from zutil.post import get_case_root
from zutil.post import calc_force_wall
from zutil.post import ProgressBar
import os

pbar = ProgressBar()

fig_list = {}
for station in station_values:
    fig_list[station] = get_figure(plt)
    fig = fig_list[station]
    span_loc = station_values[station]
    ax = fig.gca()
    set_title(ax,'$\mathbf{C_p}$ span='+str(span_loc*100)+'% \n')
    ax.grid(True)
    x_label(ax,'$\mathbf{x/c}$')
    y_label(ax,'$\mathbf{C_p}$')
    ax.axis([0.0,1.0,1.0,-1.2])
    set_ticks(ax)

    for source, colour in sources:
        plotlist_x = []
        plotlist_y = []
        for key, value in dpw5_comparative_data["L3"][source][station]['X/C'].iteritems():
            plotlist_x.append(value)
        for key, value in dpw5_comparative_data["L3"][source][station]['CP'].iteritems():
            plotlist_y.append(value)

        ax.plot(plotlist_x, plotlist_y, colour+'.', label=source)


case_name = CASE
label = 'zCFD SST L1'

status=get_status_dict(case_name)
num_procs = str(status['num processor'])

plot = 1
pbar+=5

for station in station_values:
    span_loc = station_values[station]
    fig = fig_list[station]

    ax = fig.gca()
    plot_cp_profile(ax,os.path.join(analysis.data.data_dir,get_case_root(case_name,num_procs)),
                    span_loc, label=label)
    plot += 1
    pbar += 5
        
for station in station_values:
    fig = fig_list[station]
    ax = fig.gca()
    set_legend(ax,'best')

    #fig.subplots_adjust(hspace=0.3) 
    fig.savefig(os.path.join("output", "DPW5_cp_profile_"+str(station)+".png"))

pbar.complete()
plt.show()


In [ ]:
from zutil.post import get_num_procs

def save_image(file_root, label):
    renderView1 = CreateView('RenderView')
    renderView1.ViewSize = [1080, 634]
    renderView1.InteractionMode = '2D'
    renderView1.AxesGrid = 'GridAxes3DActor'
    renderView1.OrientationAxesVisibility = 0
    renderView1.CenterOfRotation = [1327.6915283203125, 0.0, 217.05277633666992]
    renderView1.StereoType = 0
    renderView1.CameraPosition = [1327.6915283203125, 0.0, 6781.762593092753]
    renderView1.CameraFocalPoint = [1327.6915283203125, 0.0, 217.05277633666992]
    renderView1.CameraParallelScale = 1404.19167450244
    renderView1.Background = [0.0, 0.0, 0.0]
    dpw5_L1_wallpvd = PVDReader(FileName=file_root+'_wall.pvd')
    cleantoGrid1 = CleantoGrid(Input=dpw5_L1_wallpvd)
    cellDatatoPointData1 = CellDatatoPointData(Input=cleantoGrid1)
    reflect1 = Reflect(Input=cellDatatoPointData1)
    reflect1.Plane = 'Y Min'
    cpLUT = GetColorTransferFunction('cp')
    cpLUT.RGBPoints = [-1.0959302186965942, 0.231373, 0.298039, 0.752941, 0.07523328065872192, 0.865003, 0.865003, 0.865003, 1.246396780014038, 0.705882, 0.0156863, 0.14902]
    cpLUT.ScalarRangeInitialized = 1.0
    cpPWF = GetOpacityTransferFunction('cp')
    cpPWF.Points = [-1.0959302186965942, 0.0, 0.5, 0.0, 1.246396780014038, 1.0, 0.5, 0.0]
    cpPWF.ScalarRangeInitialized = 1
    reflect1Display = Show(reflect1, renderView1)
    reflect1Display.ColorArrayName = ['POINTS', 'cp']
    reflect1Display.LookupTable = cpLUT
    reflect1Display.ScalarOpacityUnitDistance = 113.80220009822894    
    Render()
    WriteImage(os.path.join("output", label+'.png'))
    
from IPython.display import Image, display

case_name = CASE
label_ = 'NASA CRM zCFD L1'
num_procs = get_num_procs(case_name)
save_image(os.path.join(analysis.data.data_dir,get_case_root(case_name,str(num_procs))),label_)
display(Image(filename=os.path.join("output", '%s.png' % label_), width=800, height=500, unconfined=True))